In [ ]:
from meta_kg.utils.py_io import *

# dynamic = read_json("./output/20221124-010221/dev_out-epoch=0_step=11250.json")
# dynamic = read_json("./output/20221129-195134/dev_out-epoch=0_step=11250.json")
dynamic = read_json("./output/20221205-051520/dev_out-epoch=0_step=3600.json")

inner_loss_token_dynamic = []
topk_tokens_dynamic = []
inner_loss_dynamic = []
for log in dynamic:
    inner_loss = log["inner_loss"]
    topk_tokens = log["topk_tokens"]
    inner_loss_token = log["inner_loss_token"]
    inner_loss_dynamic.append(inner_loss)
    topk_tokens_dynamic.append(topk_tokens)
    inner_loss_token_dynamic.append(inner_loss_token)

inner_loss_token_dynamic[0]

In [ ]:
gen_outs = [' '.join(out['gen_out'].split()[1:4]) for out in dynamic]
answers = [out['answer'] for out in dynamic]

In [ ]:
acc = 0
for i in range(len(gen_outs)):
    if gen_outs[i] == answers[i]:
        acc += 1
acc / len(gen_outs)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

index = 0


def plot_loss_token(inner_loss_token, index):
    plasma = plt.get_cmap('tab20c')
    def rescale(y): return (y - np.min(y)) / (np.max(y) - np.min(y))

    for token_loss in inner_loss_token:
        tokens = [tok for tok in list(token_loss.keys())]
        tokens.reverse()
        losses = [round(loss[0], 3) for loss in list(token_loss.values())]
        losses.reverse()

        plt.figure(figsize=(10, 2))
        plt.barh(tokens, losses, color=plasma(rescale(losses)), height=0.5)
        plt.xticks(rotation=0, fontsize=15)
        plt.yticks(rotation=0, fontsize=15)
        for i, v in enumerate(losses):
            plt.text(v, i, " "+str(v), color='blue',
                     va='center', fontweight='bold')

        # plt.savefig(f'./output/token_loss_{index}.png', bbox_inches='tight', dpi=300)
        index += 1
        plt.show()


plot_loss_token(inner_loss_token_dynamic[0], index)
plot_loss_token(inner_loss_token_dynamic[10], index)


In [ ]:
def plot_topk_token(topk_tokens, index):
    plasma = plt.get_cmap('tab20c')
    def rescale(y): return (y - np.min(y)) / (np.max(y) - np.min(y))

    for record in topk_tokens:
        orig = record[0][0]
        tokens = [tok[1] for tok in record[:5]]
        probs = [tok[2] for tok in record[:5]]

        assert(len(tokens) == len(probs))

        tokens.reverse()
        probs.reverse()
        # print(tokens)
        # print(probs)

        plt.figure(figsize=(8, 5))
        plt.barh(tokens, probs, color=plasma(rescale(probs)), height=0.5)
        plt.xticks(rotation=0, fontsize=15)
        plt.yticks(rotation=0, fontsize=15)
        plt.title(f"Original token: {orig}", fontsize=15, fontweight='bold')
        for i, v in enumerate(probs):
            plt.text(v, i, " "+str(v), color='blue',
                     va='center', fontweight='bold')
        # plt.savefig(f'./output/token_loss_{index}.png', bbox_inches='tight', dpi=300)
        index += 1
        plt.show()


index = 0
plot_topk_token(topk_tokens_dynamic[0][1], index)


In [ ]:
all_loss = []
for inner_loss_token in inner_loss_token_dynamic[:1000]:
    for token_loss in inner_loss_token:
        losses = [loss[0] for loss in list(token_loss.values())[1:]]
        for loss in losses:
            all_loss.append(loss)
all_loss = np.array(all_loss)
q25, q75 = np.percentile(all_loss, [25, 75])
bin_width = 2 * (q75 - q25) * len(all_loss) ** (-1/3)
bins = round((all_loss.max() - all_loss.min()) / bin_width)
plt.figure(figsize=(6, 4))
plt.hist(all_loss, density=False, bins=bins)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Ratio')
plt.xlabel('Loss')

In [ ]:
all_loss_diff = []
for inner_loss_token in inner_loss_token_dynamic[:1000]:
    for token_loss in inner_loss_token:
        losses = [loss[1] for loss in list(token_loss.values())[1:]]
        for loss in losses:
            all_loss_diff.append(loss)
all_loss_diff = np.array(all_loss_diff)
q25, q75 = np.percentile(all_loss_diff, [25, 75])
bin_width = 2 * (q75 - q25) * len(all_loss_diff) ** (-1/3)
bins = round((all_loss_diff.max() - all_loss_diff.min()) / bin_width)
plt.figure(figsize=(6, 4))
plt.hist(all_loss_diff, density=True, bins=bins)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Count')
plt.xlabel('Loss Difference')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

hist, xedges, yedges = np.histogram2d(
    all_loss, all_loss_diff, bins=10, range=[[0, 10], [-5, 5]])

xedges = [round(x) for x in xedges]
yedges = [round(y) for y in yedges]

cmap = sns.cm.rocket_r
fig = plt.figure(figsize=(12, 8))
ax = sns.heatmap(
        hist, linewidth=0.8, cmap=cmap,
        xticklabels=xedges[:-1], yticklabels=yedges[:-1])
plt.show()

In [ ]:
from meta_kg.utils.py_io import *

clutrr_train = read_jsonl('./data/clutrr_simple/train.jsonl')

In [ ]:
clutrr_train[0]

In [ ]:
from collections import Counter

relation_inner = [fact[1] for data in clutrr_train for fact in data['facts']]
Counter(relation_inner).most_common(10)


In [ ]:
relation_outer = [q[2] for data in clutrr_train for q in data['questions']]
Counter(relation_outer).most_common(10)

In [ ]:
import torch

checkpoint = "./output/20221205-051520/epoch=0-step=7200.ckpt"

model = torch.load(checkpoint)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
input_text = ["fact 1 123: ", "fact 2 123: ", "fact 3 123: "]
input_ids = tokenizer("fact 1 123: 1sddfgtg ", return_tensors='pt').input_ids

outputs = gpt.generate(input_ids, max_length=20, num_beams=5, do_sample=False, top_k=5, top_p=None, num_return_sequences=1)

In [ ]:
input_ids.size(1)

In [ ]:
gpt2_param = gpt.state_dict()
for param in model['state_dict']:
    if "model.model" in param:
        new_param = param.replace("model.model", "")
        gpt2_param[new_param] = model['state_dict'][param]

In [ ]:
[tokenizer.decode(out, skip_special_tokens=True) for out in outputs]

In [35]:
from transformers import GenerationConfig

max_length = 20
generation_config = GenerationConfig.from_pretrained(
    "distilgpt2",
    num_beams=5,
    early_stopping=True,
    top_p=0.5,
    top_k=5,
    do_sample=True,
    num_return_sequences=1
)

In [24]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id
inputs = tokenizer(["Today is"], return_tensors="pt")

In [36]:
model.generation_config = generation_config

In [37]:
outputs = model.generate(input_ids=inputs.input_ids, max_length=20)

In [38]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Today is a new feature for the Windows Phone 8.1 operating system.\n\n\n\n\n'

In [1]:
class Node:
    def __init__(self, id):
        self.id = id
        self.neighbors = []
        self.slice = None
        self.random_number = None

    def add_neighbor(self, node):
        self.neighbors.append(node)

    def set_slice(self, slice):
        self.slice = slice

    def set_random_number(self, random_number):
        self.random_number = random_number

    def __str__(self):
        return f"Node {self.id}"

    def __repr__(self):
        return self.__str__()
    
nodes = [Node(i) for i in range(20)]

for i in nodes:
    i.neighbors = {}
    i.slice = (i.random_number / (20/10))
    j = random.choice(i.neighbors.keys())
    send_gossip(i, j, randomly selected node from slice i.slice)

// Gossip phase
for round = 1 to num_rounds:
    for each node i:
        j = randomly selected neighbor of i
        k = randomly selected node from slice i.slice
        send_gossip(i, j, k)
        
        if j.slice == i.slice:
            l = randomly selected neighbor of j
            send_gossip(j, i, l)

// Termination phase
for each node i:
    stop_gossiping(i)

SyntaxError: invalid syntax (514494627.py, line 24)

In [4]:
import random

random.randint(0, 10) / (20 / 10)

1.5